<h1>Calculating the grounded extensions of a graph using Matrices</h1>

We assume a that arguments attack relations are given in matrix form, in the form of a confusion matrix. We'll start by creating the matrix form of the argument graph below:

In [1]:
import numpy as np
import pandas as pd

In [2]:
dungGraph = np.array([[0,0,1], [1,0,0], [0,0,0]])

In [3]:
dungGraph

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 0]])

In [4]:
sumArgs = dungGraph.sum(axis=0)

In [5]:
inArgs = np.argwhere(sumArgs == 0)
inArgs = (inArgs.tolist())
inArgs = [i[0] for i in inArgs]
print(inArgs)

[1]


In [6]:
a = dungGraph[inArgs, :]
print(a)

[[1 0 0]]


In [7]:
outs = (np.unique(np.where(a>0)[1])).tolist()
print(outs)

[0]


<h3>Function for Calculating the In and Out Arguments of a Single Iteration</h3>

In [8]:
def getInOutArgs(argMtx):
    sumArgs = argMtx.sum(axis=0)
    
    inArgs = np.argwhere(sumArgs == 0)
    inArgs = (inArgs.tolist())
    inArgs = [i[0] for i in inArgs]
    
    attacked = argMtx[inArgs, :]
    
    outArgs = (np.unique(np.where(attacked>0)[1])).tolist()
    
    return inArgs, outArgs

<h3>Complete Function for Calculating the Grounded Extension</h3>

In [9]:
def calculateGroundedExtension(argMtx):
    argTypes = np.array(range(0, argMtx.shape[0]))

    ext = []
    terminate = False

    while not terminate:
        inArgs, outArgs = getInOutArgs(argMtx)

        if len(inArgs) > 0:
            ext.extend(list(argTypes[inArgs]))
            argsDelete = inArgs + outArgs
            argMtx = np.delete(argMtx, argsDelete, axis = 0)
            argMtx = np.delete(argMtx, argsDelete, axis = 1)
            argTypes = np.delete(argTypes, argsDelete)

        else:
            break

        sums = np.sum(argMtx.sum(axis=0))
        
        # If we find that the resulting graph (having deleted current in and out args) is got no more attacks in it then add 
        # whatever is leftover to the extension
        if sums == 0:
            ext.extend(list(argTypes))
            terminate = True
        
    return ext

argMtx = np.array([[0,1], [1,0]])
print(calculateGroundedExtension(argMtx))

[]


In [10]:
argMtx = np.array([[0,0,1], [1,0,0], [0,0,0]])
argTypes = np.array(range(0, argMtx.shape[0]))

ext = []
terminate = False

while not terminate:
    inArgs, outArgs = getInOutArgs(argMtx)
    
    if len(inArgs) > 0:
        ext.extend(list(argTypes[inArgs]))
        argsDelete = inArgs + outArgs
        argMtx = np.delete(argMtx, argsDelete, axis = 0)
        argMtx = np.delete(argMtx, argsDelete, axis = 1)
        argTypes = np.delete(argTypes, argsDelete)
        
    else:
        break
    
    sums = np.sum(dungGraph.sum(axis=0))
    terminate = (sums == 0)
        

print('extension')
print(ext)
    

extension
[1, 2]


<h1>Generating All Attack Relations Given a Review and Rating</h1>


In [11]:
import itertools
posArgs = [0]
negArgs = [1,2]

nargs = len(posArgs + negArgs)
rating = 1

groundedExtension = negArgs

In [12]:
allPermutations = np.array(list(itertools.product([0,1], repeat=nargs*nargs)))


circularAttacks = (np.arange(0, nargs*nargs, nargs+1)).tolist()

samePolarityAttacks = []
posList = list(itertools.permutations(posArgs, 2))
negList = list(itertools.permutations(negArgs, 2))
totList = posList + negList

for l in totList:
    ptn = ((l[0]+1)*nargs)- ((nargs+1) - l[1]) + 1 # All the odd +1 are to account for the shift in 0 index
    print('ptn')
    print(l)
    print(ptn)
    samePolarityAttacks.append(ptn)

    
graphsToDelete = np.unique([circularAttacks + samePolarityAttacks])

print(graphsToDelete)
subGraphs = allPermutations[:,graphsToDelete]
cutDownGraphs = np.delete(allPermutations, (np.where(subGraphs>0)[0]).tolist(), axis = 0)

print(cutDownGraphs)

ptn
(1, 2)
5
ptn
(2, 1)
7
[0 4 5 7 8]
[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 1 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 1 0 0]
 [0 0 1 1 0 0 0 0 0]
 [0 0 1 1 0 0 1 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 1 0 0]
 [0 1 0 1 0 0 0 0 0]
 [0 1 0 1 0 0 1 0 0]
 [0 1 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 1 0 0]
 [0 1 1 1 0 0 0 0 0]
 [0 1 1 1 0 0 1 0 0]]


In [13]:
first = cutDownGraphs[3]
print(first)

[0 0 0 1 0 0 1 0 0]


<h3>Convert Each Member of CutDown into an Attack Matrix</h3>

In [14]:
attackMatrix = np.vstack( np.array_split(np.array(first), nargs))
print(attackMatrix)

[[0 0 0]
 [1 0 0]
 [1 0 0]]


<h3>Iterate Over Each Permutation and See Grounded Ext</h3>

We are checking to see that the grounded extension of each graph matches our ideal grounded extension. If we want we can change this criteria to say that the grounded extension of each graph should be a subset of the ideal grounded extension. This way it means that we allow some arguments to be attacked. Meaning arguments of the same set do not defend eachother all the time

In [15]:
acceptedGraphs = []

for graph in cutDownGraphs:
    attMtx = np.vstack( np.array_split(np.array(graph), nargs))
    ext = calculateGroundedExtension(attMtx)
    if set(groundedExtension) == set(ext): 
        acceptedGraphs.append(graph.tolist())    

In [16]:
acceptedGraphs = np.array(acceptedGraphs)

sumOfAttacks = (acceptedGraphs).sum(axis=0)
print(np.vstack( np.array_split(np.array(sumOfAttacks), nargs)))

[[0 2 2]
 [5 0 0]
 [5 0 0]]


In [17]:
normalisedAttcks = sumOfAttacks / acceptedGraphs.shape[0]
print(normalisedAttcks)

[ 0.          0.28571429  0.28571429  0.71428571  0.          0.
  0.71428571  0.          0.        ]


<h3>Function for Calculating Normalised Distribution</h3>

In [18]:
def calculateProbabilityDistribution(posArgs, negArgs, rating):
    
    nargs = len(posArgs + negArgs)
    allPermutations = np.array(list(itertools.product([0,1], repeat=nargs*nargs)))
    
    # Determine the Polarity of the Post
    if rating < 5:
        groundedExtension = negArgs
    elif rating > 6:
        groundedExtension = posArgs
    else:
        groundedExtension = []
    
    
    # Create list of Attacks that we will never need. Circular attacks, and attacks between arguments of same polarity
    circularAttacks = (np.arange(0, nargs*nargs, nargs+1)).tolist()
    samePolarityAttacks = []
    posList = list(itertools.permutations(posArgs, 2))
    negList = list(itertools.permutations(negArgs, 2))
    totList = posList + negList

    for l in totList:
        ptn = ((l[0]+1)*nargs)- ((nargs+1) - l[1]) + 1 # All the odd +1 are to account for the shift in 0 index
        samePolarityAttacks.append(ptn)

    
    graphsToDelete = np.unique([circularAttacks + samePolarityAttacks])
    subGraphs = allPermutations[:,graphsToDelete]
    cutDownGraphs = np.delete(allPermutations, (np.where(subGraphs>0)[0]).tolist(), axis = 0)

    # Iterate over the set of graphs that are possible (excluding circular attacks and same polarity attacks) to see which one's 
    # have a grounded extension matching the polarti
    acceptedGraphs = []
    for graph in cutDownGraphs:
        attMtx = np.vstack( np.array_split(np.array(graph), nargs))
        ext = calculateGroundedExtension(attMtx)
        if set(groundedExtension) == set(ext): 
            acceptedGraphs.append(graph.tolist()) 
        
    # Aggregate then normalise the complete set of Attacks
    acceptedGraphs = np.array(acceptedGraphs)
    sumOfAttacks = (acceptedGraphs).sum(axis=0)
    normalisedAttcks = sumOfAttacks / acceptedGraphs.shape[0]
    
    return normalisedAttcks

<h1>Generating Artificial Data</h1>

We will generate a bunch of artificial data to test our pipeline with

In [19]:
highestRating = 10
noArgsTypes = 6
positiveArgTypes = [0,1,2]
negativeArgTypes = [3,4,5]
noPosts = 5

randomReviews = np.random.randint(noArgsTypes, size=(noPosts,4))
randomRatings = np.random.randint(highestRating, size=(noPosts,1))

globalGraph = np.zeros(noArgsTypes*noArgsTypes)

<h3>Calculating Normal Distributions for Each Review</h3>

In [20]:
for idx, review in enumerate(randomReviews):
    print('-----')
    print('review: ', review)
    print('rating: ', randomRatings[idx])
    
    posArgs = list(set(positiveArgTypes).intersection(review.tolist()))
    print('posArgs: ', posArgs)
    negArgs = list(set(negativeArgTypes).intersection(review.tolist()))
    print('negArgs: ', negArgs)
    posArgsIdx = list(range(0,len(posArgs)))
    negArgsIdx = list(range(len(posArgs), len(posArgs)+len(negArgs)))
    
    nmlDstb = calculateProbabilityDistribution(posArgsIdx, negArgsIdx, randomRatings[idx])
    print(nmlDstb)
    globalContribution = np.zeros(noArgsTypes*noArgsTypes)
    
    noOfArgTypesFound = len(list(posArgs+negArgs))
    
    allAttacks = list(itertools.product(list(posArgs+negArgs), repeat=2))
    allAttacksIdx = []
    for attack in allAttacks:
        ptn = ((attack[0]+1)*noArgsTypes)- ((noArgsTypes+1) - attack[1])
        allAttacksIdx.append(ptn)
    
    globalContribution[allAttacksIdx] = nmlDstb
    
    globalGraph = np.add(globalGraph, globalContribution)
    
 
print('-----------------------------------------------------------------')
print(globalGraph)

-----
review:  [2 5 1 1]
rating:  [7]
posArgs:  [1, 2]
negArgs:  [5]
[ 0.          0.          0.71428571  0.          0.          0.71428571
  0.28571429  0.28571429  0.        ]
-----
review:  [2 4 1 0]
rating:  [1]
posArgs:  [0, 1, 2]
negArgs:  [4]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  0.]
-----
review:  [0 4 1 1]
rating:  [5]
posArgs:  [0, 1]
negArgs:  [4]
[ 0.          0.          0.66666667  0.          0.          0.66666667
  1.          1.          0.        ]
-----
review:  [0 4 0 0]
rating:  [3]
posArgs:  [0]
negArgs:  [4]
[ 0.  0.  1.  0.]
-----
review:  [1 5 5 1]
rating:  [7]
posArgs:  [1]
negArgs:  [5]
[ 0.  1.  0.  0.]
-----------------------------------------------------------------
[ 0.          0.          0.          0.66666667  0.          0.          0.
  0.          0.          0.66666667  1.71428571  0.          0.          0.
  0.          0.          0.71428571  0.          0.          0.          0.
  0.          0.          3.         

In [21]:
from sklearn.utils.extmath import cartesian